## Summary

This is a notebook where we will compute the capitalization rates for certain buildings in New York City. The data are based on public information about property sales prices, and tax information on the incomes and expenses for those buildings obtained by **FOIL**. The first step is to import the relevant data from spreadsheets in the [data](../data) folder. After that, the two spreadsheets are cleaned by removing duplicates and empty cells, and merged according to the unique identifiers for Borough, block and lot. 

Finally, we calculate the capitalization rates for each building (using the formula *CR=Income-Expenses/Building Cost*) and compute the average values (mean and median) for each borough. The results are exported to spreadsheets in the [output](../output) folder. 

In [1]:
import pandas as pd


We start by creating a dataframe called *rawacris*, populated by the data on building sales from 2020-2022.

In [2]:
rawacris=pd.read_csv('../data/acris_sales_2020-2022.csv', converters={'bbl':str, "document_id": str})
rawacris

,bbl,document_id,res_unit,total_units_doc,prop_units,document_amt,price_per_blgd,ppu,percent_trans,units_bought,...,z_percent_trans_ppu,boro,year,p1_count,p1_a_name,p1_a_address_1,p1_a_address_2,p1_a_city,p1_a_state,p1_a_zip
0,2032920019,2020080400603001,31,31,1.0,4950000,4950000,159677,100.0,31.0,...,-0.206,2,2020,1,2968 PERRY LLC,C/O: THE MORGAN GROUP,"ONE SOUND SHORE DRIVE, SUITE 203",GREENWICH,CT,6830
1,1019370046,2020080600869001,8,8,1.0,800000,800000,100000,100.0,8.0,...,-0.707,1,2020,2,"RONALD ADAMS, AS ADMINISTRATOR",234 W 132ND ST,NaN,NEW YORK,NY,100277804
2,1004350011,2020090100498001,10,10,1.0,8500000,8500000,850000,100.0,10.0,...,0.815,1,2020,1,94 ST MARKS LLC,C/O: PENN SOUTH CAPITAL,"41 MADISON AVE, 31ST FLOOR",NEW YORK,NY,10010
3,2039230054,2020091400652001,17,17,1.0,2200000,2200000,129412,100.0,17.0,...,-0.567,2,2020,1,1532 LELAND AVENUE INC,"35 E GRASSY SPRAIN RD, STE. 400","C/O VINCENT CASTELLANO, ESQ",YONKERS,NY,10710
4,1004180047,2020092100801001,22,22,1.0,5350000,5350000,243182,100.0,22.0,...,-0.417,1,2020,1,109 ELDRIDGE LLC,"C/O KOSSOFF, PLLC","217 BROADWAY, SUITE 401",NEW YORK,NY,10007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,3018510021,2021101100509001,7,7,1.0,1050000,1050000,150000,100.0,7.0,...,-0.824,3,2021,11,"GROSVENOR, FELICIA",82 MACDONOUGH STREET,NaN,BROOKLYN,NY,112162205
1999,3078190046,2021110200141002,83,83,1.0,6733800,6733800,81130,60.0,49.8,...,-0.885,3,2021,1,"FRED & IVAN LEIST SPECIAL K, LLC",1710 CORTELYOU ROAD,NaN,BROOKLYN,NY,11226
2000,3033860058,2021121701208001,6,6,1.0,975000,975000,162500,100.0,6.0,...,-0.772,3,2021,2,"CHIAFFITELLI, NANCY B",1517 JEFFERSON AVENUE,NaN,BROOKLYN,NY,11237
2001,3023110019,2021122800426001,5,5,1.0,4000000,4000000,800000,100.0,5.0,...,1.862,3,2021,1,THE NORTH NINTH STREET GROUP LLC,164 CLYMER STREET,NaN,BROOKLYN,NY,11211


There are some duplicate values in the raw acris files. To eliminate them, we will sort by year, and use only the most recent rows.

In [3]:
acris=rawacris.sort_values(by=["year","bbl"])
sales=acris.drop_duplicates(subset=["bbl"],keep="last")

In [4]:
sales

,bbl,document_id,res_unit,total_units_doc,prop_units,document_amt,price_per_blgd,ppu,percent_trans,units_bought,...,z_percent_trans_ppu,boro,year,p1_count,p1_a_name,p1_a_address_1,p1_a_address_2,p1_a_city,p1_a_state,p1_a_zip
418,1001360020,2020112400588001,5,5,1.0,3717900,3717900,743580,100.0,5.0,...,0.599,1,2020,1,124 CHAMBERS STREET PARTNERS LLC,"C/O: MNM PROPERTIES, LLC",300 EAST 76TH STREET; APT A2,NEW YORK,NY,10021
69,1002050020,2020020600408001,11,11,1.0,3820000,3820000,347273,100.0,11.0,...,-0.205,1,2020,1,"SHAY, AS REFEREE, ELAINE","800 THIRD AVENUE, SUITE 2800",NaN,NEW YORK,NY,10022
761,1002740013,2020042900097001,16,16,1.0,7875000,7875000,492188,100.0,16.0,...,0.089,1,2020,1,73 MONROE HOLDINGS LLC,"80 FIFTH AVENUE, SUITE 1201",NaN,NEW YORK,NY,10011
589,1002800054,2020032700678001,15,30,0.5,14000000,7000000,466667,100.0,15.0,...,0.037,1,2020,1,CATHERINE STREET ASSOCIATES LLC,"C/O: R.A. COHEN & ASSOCIATES, INC",60 E 42ND ST,NEW YORK,NY,101650006
588,1002800055,2020032700678001,15,30,0.5,14000000,7000000,466667,100.0,15.0,...,0.037,1,2020,1,CATHERINE STREET ASSOCIATES LLC,"C/O: R.A. COHEN & ASSOCIATES, INC",60 E 42ND ST,NEW YORK,NY,101650006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,4161550012,2021061702334001,10,10,1.0,975000,975000,97500,100.0,10.0,...,-0.866,4,2021,1,M & M MANAGEMENT GROUP LLC,2671 KENNY AVENUE,NaN,MERRICK,NY,11566
846,4161690065,2021092400376001,10,10,1.0,2900000,2900000,290000,100.0,10.0,...,0.247,4,2021,1,ADMIRAL PROPERTY GROUP LLC,6833 SHORE ROAD,NaN,BROOKLYN,NY,11220
1521,4162290071,2021121701263001,11,11,1.0,655000,655000,59545,100.0,11.0,...,-1.086,4,2021,1,"FUHRMAN, MIMI","39 GRAMERCY PARK NORTH, #7B",NaN,NEW YORK,NY,10010
465,4162310055,2021010400256002,5,5,1.0,455000,455000,91000,100.0,5.0,...,-0.904,4,2021,3,"CHIARAPPA, FRANK",.,NaN,.,FL,11111


Next, we create a new dataframe called taxdata. This will be populated with information from the applications by building owners for corrections to their tax payments.

In [5]:
taxdata=pd.read_csv('../data/file_201_data_sm_tract.csv', dtype={"BORO":str, "BLOCK":str, "FROM_LOT":str})
taxdata

,BORO,BLOCK,FROM_LOT,TO_LOT,FILING YEAR,REGULATED,UNREGULATED,OFFICE,OWNER/RELATED SPACE,GOV. RENTAL SUBSIDIES,TOTAL INCOME FROM REAL ESTATE,TOTAL EXPENSES,AMORTIZED LEASE AND TENANT IMP. COSTS,REPAIRS AND MAINT.,tract_10
0,1,00447,0025,NaN,2021,3674.0,NaN,NaN,NaN,NaN,93074.0,96825.0,NaN,28456.0,NaN
1,1,01079,0061,NaN,2021,NaN,NaN,NaN,NaN,NaN,295046.0,15020.0,NaN,7709.0,NaN
2,1,01000,0029,NaN,2021,NaN,NaN,129305065.0,NaN,NaN,159048807.0,39952316.0,11508372.0,5936267.0,NaN
3,1,01000,0061,NaN,2021,NaN,NaN,NaN,NaN,714647.0,714647.0,1236523.0,NaN,62443.0,NaN
4,1,01001,1001,1001.0,2021,NaN,NaN,NaN,NaN,NaN,3625614.0,296800.0,76509.0,7819.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26881,5,03960,0065,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN
26882,5,03960,0007,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN
26883,5,03960,0061,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN
26884,5,03960,0065,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN


Note that taxdata does not have a column for BBL. We can fix that by creating a new column combining the entries for Boro, block, and From_lot

In [6]:
taxdata["bbl"]=taxdata["BORO"]+taxdata["BLOCK"]+taxdata["FROM_LOT"]
taxdata

,BORO,BLOCK,FROM_LOT,TO_LOT,FILING YEAR,REGULATED,UNREGULATED,OFFICE,OWNER/RELATED SPACE,GOV. RENTAL SUBSIDIES,TOTAL INCOME FROM REAL ESTATE,TOTAL EXPENSES,AMORTIZED LEASE AND TENANT IMP. COSTS,REPAIRS AND MAINT.,tract_10,bbl
0,1,00447,0025,NaN,2021,3674.0,NaN,NaN,NaN,NaN,93074.0,96825.0,NaN,28456.0,NaN,1004470025
1,1,01079,0061,NaN,2021,NaN,NaN,NaN,NaN,NaN,295046.0,15020.0,NaN,7709.0,NaN,1010790061
2,1,01000,0029,NaN,2021,NaN,NaN,129305065.0,NaN,NaN,159048807.0,39952316.0,11508372.0,5936267.0,NaN,1010000029
3,1,01000,0061,NaN,2021,NaN,NaN,NaN,NaN,714647.0,714647.0,1236523.0,NaN,62443.0,NaN,1010000061
4,1,01001,1001,1001.0,2021,NaN,NaN,NaN,NaN,NaN,3625614.0,296800.0,76509.0,7819.0,NaN,1010011001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26881,5,03960,0065,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN,5039600065
26882,5,03960,0007,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN,5039600007
26883,5,03960,0061,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN,5039600061
26884,5,03960,0065,NaN,2021,NaN,NaN,NaN,NaN,NaN,2274290.0,659462.0,59866.0,242514.0,NaN,5039600065


Once again there are some duplicates, and it's not clear why. The simplest solution is to create a new DF that omits all duplicate entries:

In [7]:
refinedtaxes=taxdata.drop_duplicates(subset=["bbl"], keep=False)
refinedtaxes

,BORO,BLOCK,FROM_LOT,TO_LOT,FILING YEAR,REGULATED,UNREGULATED,OFFICE,OWNER/RELATED SPACE,GOV. RENTAL SUBSIDIES,TOTAL INCOME FROM REAL ESTATE,TOTAL EXPENSES,AMORTIZED LEASE AND TENANT IMP. COSTS,REPAIRS AND MAINT.,tract_10,bbl
0,1,00447,0025,NaN,2021,3674.0,NaN,NaN,NaN,NaN,93074.0,96825.0,NaN,28456.0,NaN,1004470025
1,1,01079,0061,NaN,2021,NaN,NaN,NaN,NaN,NaN,295046.0,15020.0,NaN,7709.0,NaN,1010790061
2,1,01000,0029,NaN,2021,NaN,NaN,129305065.0,NaN,NaN,159048807.0,39952316.0,11508372.0,5936267.0,NaN,1010000029
3,1,01000,0061,NaN,2021,NaN,NaN,NaN,NaN,714647.0,714647.0,1236523.0,NaN,62443.0,NaN,1010000061
4,1,01001,1001,1001.0,2021,NaN,NaN,NaN,NaN,NaN,3625614.0,296800.0,76509.0,7819.0,NaN,1010011001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26875,5,00942,0132,NaN,2021,NaN,120635.0,NaN,NaN,NaN,120635.0,83213.0,NaN,14348.0,NaN,5009420132
26876,5,00947,0001,NaN,2021,NaN,NaN,36675.0,200250.0,NaN,236925.0,47919.0,2525.0,5789.0,NaN,5009470001
26877,5,00955,0075,NaN,2021,1387527.0,NaN,NaN,NaN,NaN,1404486.0,1361772.0,NaN,280776.0,3.608502e+10,5009550075
26878,5,03208,0020,NaN,2021,NaN,NaN,NaN,NaN,NaN,306866.0,NaN,NaN,NaN,NaN,5032080020


Now it's time to combine the two dataframes, joined by the shared values for bbl.

In [8]:
combined=pd.merge(sales, refinedtaxes, on="bbl", how="inner")

In [9]:
combined


,bbl,document_id,res_unit,total_units_doc,prop_units,document_amt,price_per_blgd,ppu,percent_trans,units_bought,...,REGULATED,UNREGULATED,OFFICE,OWNER/RELATED SPACE,GOV. RENTAL SUBSIDIES,TOTAL INCOME FROM REAL ESTATE,TOTAL EXPENSES,AMORTIZED LEASE AND TENANT IMP. COSTS,REPAIRS AND MAINT.,tract_10
0,1002800054,2020032700678001,15,30,0.5,14000000,7000000,466667,100.00,15.0000,...,110000.0,540000.0,NaN,NaN,NaN,802910.0,185578.0,42008.0,23010.0,NaN
1,1004000042,2020062200670002,22,22,1.0,2500000,2500000,113636,100.00,22.0000,...,92777.0,NaN,NaN,NaN,NaN,98406.0,89730.0,128.0,5506.0,NaN
2,1004010013,2020082400793001,14,14,1.0,3950000,3950000,282143,100.00,14.0000,...,281610.0,NaN,NaN,NaN,NaN,281610.0,83456.0,17100.0,6876.0,NaN
3,1004070054,2020091100102001,27,27,1.0,2800000,2800000,103704,100.00,27.0000,...,184499.0,NaN,NaN,NaN,NaN,222461.0,230041.0,NaN,43687.0,NaN
4,1004090003,2020011300450001,15,15,1.0,9080000,9080000,605333,100.00,15.0000,...,NaN,309530.0,NaN,NaN,NaN,349530.0,148392.0,NaN,14208.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,3074220917,2021010400962001,113,113,1.0,850000,850000,7522,4.25,4.8025,...,1452133.0,NaN,NaN,NaN,81205.0,1534213.0,642856.0,NaN,132715.0,NaN
247,3074640022,2021010400812001,86,86,1.0,950000,950000,11047,6.79,5.8394,...,1297985.0,NaN,18280.0,NaN,13811.0,1331333.0,436382.0,NaN,106240.0,NaN
248,4006230063,2021052800831001,17,17,1.0,3100000,3100000,182353,100.00,17.0000,...,130117.0,NaN,NaN,NaN,NaN,130117.0,39147.0,NaN,27168.0,NaN
249,4032560031,2021050401271001,13,13,1.0,4250000,4250000,326923,100.00,13.0000,...,14800.0,92797.0,NaN,NaN,NaN,157117.0,47009.0,NaN,NaN,NaN


Some of these entries may have NaN values for income, expenses, or building price. The next step is to eliminate those:


In [10]:
combined=combined.dropna(subset=["TOTAL INCOME FROM REAL ESTATE","TOTAL EXPENSES","price_per_blgd"])

Next, we make a new column called "cap rate", which calculates the capitalization rate based on income, expenses, and building price.

In [11]:
combined["cap rate"]=(combined["TOTAL INCOME FROM REAL ESTATE"]-combined["TOTAL EXPENSES"])/combined["price_per_blgd"]

/var/folders/lw/9fwsvtm56_d6lkb4v5m8nyvm0000gn/T/ipykernel_65206/3890630092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined["cap rate"]=(combined["TOTAL INCOME FROM REAL ESTATE"]-combined["TOTAL EXPENSES"])/combined["price_per_blgd"]


We don't need to follow all 40 columns of data. Instead we create a new list called [columns] representing the most important ones:

In [12]:
columns=["BORO","bbl","price_per_blgd","TOTAL INCOME FROM REAL ESTATE","TOTAL EXPENSES","cap rate"]

In [13]:
combined[columns]

,BORO,bbl,price_per_blgd,TOTAL INCOME FROM REAL ESTATE,TOTAL EXPENSES,cap rate
0,1,1002800054,7000000,802910.0,185578.0,0.088190
1,1,1004000042,2500000,98406.0,89730.0,0.003470
2,1,1004010013,3950000,281610.0,83456.0,0.050166
3,1,1004070054,2800000,222461.0,230041.0,-0.002707
4,1,1004090003,9080000,349530.0,148392.0,0.022152
...,...,...,...,...,...,...
246,3,3074220917,850000,1534213.0,642856.0,1.048655
247,3,3074640022,950000,1331333.0,436382.0,0.942054
248,4,4006230063,3100000,130117.0,39147.0,0.029345
249,4,4032560031,4250000,157117.0,47009.0,0.025908


Now it's time to do some analysis. Here are the average cap rates for properties bought in 2020-2021, sorted by borough:

In [14]:
means=combined[columns].groupby("BORO").mean()
means

,bbl,price_per_blgd,TOTAL INCOME FROM REAL ESTATE,TOTAL EXPENSES,cap rate
BORO,,,,,
1,inf,1.325226e+07,762149.438462,446667.269231,0.034488
2,inf,4.841434e+06,470606.727273,291167.878788,0.031378
3,inf,1.014282e+07,774816.878788,323546.878788,0.124138
4,3339050022834408889981189284793852384591318190...,7.631583e+06,658423.750000,306392.666667,0.030842


And here's the same data, sorted by median cap rate:

In [15]:
medians=combined[columns].groupby("BORO").median()
medians

,bbl,price_per_blgd,TOTAL INCOME FROM REAL ESTATE,TOTAL EXPENSES,cap rate
BORO,,,,,
1,1013205038.5,5837500.0,306500.0,154119.0,0.021886
2,2031030020.0,3960000.0,434518.0,252763.0,0.032705
3,3021520010.5,4175000.0,330312.5,155957.5,0.037514
4,4031675058.5,4517500.0,304525.5,138782.0,0.037327


In [16]:
combined[columns].to_csv('../output/capitalizationrates.csv')

And just in case you need that to be more clear, here's a dataframe showing the averages by borough. We'll start with a little renaming and then create a new dataframe:



In [17]:
means["mean cap rate(%)"]=100*means["cap rate"]
medians["median cap rate(%)"]=100*medians["cap rate"]
medians["Borough Name"]=["Manhattan","Bronx","Brooklyn","Queens"]
Averages=pd.concat([means, medians], axis=1)
cols=["Borough Name","median cap rate(%)","mean cap rate(%)"]


In [18]:
Averages[cols]

,Borough Name,median cap rate(%),mean cap rate(%)
BORO,,,
1,Manhattan,2.188566,3.448764
2,Bronx,3.270514,3.137758
3,Brooklyn,3.751361,12.413763
4,Queens,3.732732,3.084247


In [19]:
Averages[cols].to_csv("../output/ResultsbyBoro.csv")